In [1]:
import pandas as pd
import sqlite3

print("Libraries imported successfully")


Libraries imported successfully


In [2]:
orders = pd.read_csv("orders.csv")

print("Orders data loaded")
orders.head()


Orders data loaded


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
users = pd.read_json("users.json")

print("Users data loaded")
users.head()


Users data loaded


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
# Create SQLite database in memory
conn = sqlite3.connect("restaurants.db")
cursor = conn.cursor()

# Read SQL file
with open("restaurants.sql", "r", encoding="utf-8") as f:
    sql_script = f.read()

# Execute SQL script
cursor.executescript(sql_script)

# Load restaurants table into pandas
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

print("Restaurants data loaded")
restaurants.head()


Restaurants data loaded


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
restaurants.columns


Index(['restaurant_id', 'restaurant_name', 'cuisine', 'rating'], dtype='object')

In [6]:
df = orders.merge(users, on="user_id", how="left") \
           .merge(restaurants, on="restaurant_id", how="left")

print("Final dataset created")
df.head()


Final dataset created


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
# Drop duplicate restaurant name column
df.drop(columns=["restaurant_name_x"], inplace=True)

# Rename restaurant_name_y to restaurant_name
df.rename(columns={"restaurant_name_y": "restaurant_name"}, inplace=True)

# Check final columns
df.columns


Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'name', 'city', 'membership', 'restaurant_name', 'cuisine', 'rating'],
      dtype='object')

In [8]:
df.to_csv("final_food_delivery_dataset.csv", index=False)

print("final_food_delivery_dataset.csv saved successfully")


final_food_delivery_dataset.csv saved successfully


In [9]:
# Convert order_date from string to datetime
df["order_date"] = pd.to_datetime(df["order_date"], format="%d-%m-%Y")

# Check data types
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         10000 non-null  int64         
 1   user_id          10000 non-null  int64         
 2   restaurant_id    10000 non-null  int64         
 3   order_date       10000 non-null  datetime64[ns]
 4   total_amount     10000 non-null  float64       
 5   name             10000 non-null  object        
 6   city             10000 non-null  object        
 7   membership       10000 non-null  object        
 8   restaurant_name  10000 non-null  object        
 9   cuisine          10000 non-null  object        
 10  rating           10000 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(5)
memory usage: 859.5+ KB


In [11]:
# Save final dataset after fixing dates
df.to_csv("final_food_delivery_dataset.csv", index=False)
print("Final dataset updated with datetime order_date")


Final dataset updated with datetime order_date


In [12]:
# Filter only Gold members
gold_df = df[df["membership"] == "Gold"]

# Group by city and sum total_amount
city_revenue = gold_df.groupby("city")["total_amount"].sum().sort_values(ascending=False)

city_revenue


city
Chennai      1080909.79
Pune         1003012.32
Bangalore     994702.59
Hyderabad     896740.19
Name: total_amount, dtype: float64

In [13]:
# Group by cuisine and calculate average order value
avg_order_cuisine = df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)
avg_order_cuisine


cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: total_amount, dtype: float64

In [14]:
# Total spend per user
user_total = df.groupby("user_id")["total_amount"].sum()

# Count users with total spend > 1000
high_spenders_count = (user_total > 1000).sum()
high_spenders_count


np.int64(2544)

In [15]:
# Create rating bins
rating_bins = [3.0, 3.5, 4.0, 4.5, 5.0]
rating_labels = ["3.0-3.5", "3.6-4.0", "4.1-4.5", "4.6-5.0"]

# Group by rating range and sum revenue
df['rating_range'] = pd.cut(df['rating'], bins=rating_bins, labels=rating_labels, include_lowest=True)
rating_revenue = df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)
rating_revenue


C:\Users\Madan\AppData\Local\Temp\ipykernel_20868\3509792551.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_revenue = df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False)


rating_range
4.6-5.0    2197030.75
3.0-3.5    2136772.70
4.1-4.5    1960326.26
3.6-4.0    1717494.41
Name: total_amount, dtype: float64

In [16]:
# Filter Gold members
gold_df = df[df["membership"] == "Gold"]

# Group by city and calculate average order value
gold_city_avg = gold_df.groupby("city")["total_amount"].mean().sort_values(ascending=False)
gold_city_avg


city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: total_amount, dtype: float64

In [ ]:
# Group by cuisine: number of restaurants & total revenue
cuisine_stats = df.groupby("cuisine").agg(
    num_restaurants=("restaurant_id","nunique"),
    total_revenue=("total_amount","sum")
).sort_values(["num_restaurants","total_revenue"])
cuisine_stats


,num_restaurants,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [18]:
# Percentage of orders by Gold members
gold_pct = (df[df["membership"]=="Gold"].shape[0] / df.shape[0]) * 100
round(gold_pct)


50

In [19]:
# Group by restaurant: average order value & total orders
restaurant_stats = df.groupby("restaurant_name").agg(
    avg_order=("total_amount","mean"),
    total_orders=("order_id","count")
).query("total_orders < 20").sort_values("avg_order", ascending=False)

restaurant_stats.head(10)


,avg_order,total_orders
restaurant_name,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16
Restaurant_298,989.822000,15
Restaurant_56,989.467222,18
Restaurant_135,988.702222,18
Restaurant_343,986.026471,17


In [21]:
# Group by membership and cuisine, sum total_amount
combo_revenue = df.groupby(["membership","cuisine"])["total_amount"].sum().sort_values(ascending=False)
combo_revenue


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [ ]:
# Make sure order_date is datetime (you did this already)
df["order_date"] = pd.to_datetime(df["order_date"], format="%d-%m-%Y")

# Create quarter column
df["quarter"] = df["order_date"].dt.to_period("Q")

# Check it
df[["order_date","quarter"]].head()


,order_date,quarter
0,2023-02-18,2023Q1
1,2023-01-18,2023Q1
2,2023-07-15,2023Q3
3,2023-10-04,2023Q4
4,2023-12-25,2023Q4


In [24]:
quarter_revenue = df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False)
quarter_revenue


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64

In [25]:
gold_orders_count = df[df["membership"] == "Gold"].shape[0]
gold_orders_count


4987

In [26]:
hyderabad_revenue = df[df["city"]=="Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)


1889367

In [27]:
distinct_users = df["user_id"].nunique()
distinct_users


2883

In [28]:
avg_gold_aov = df[df["membership"]=="Gold"]["total_amount"].mean()
round(avg_gold_aov, 2)


np.float64(797.15)

In [29]:
high_rating_orders = df[df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [30]:
# First find top revenue city for Gold members
gold_city_revenue = df[df["membership"]=="Gold"].groupby("city")["total_amount"].sum()
top_gold_city = gold_city_revenue.idxmax()

# Now count orders in that city among Gold members
top_gold_city_orders = df[(df["membership"]=="Gold") & (df["city"]==top_gold_city)].shape[0]
top_gold_city_orders


1337